# Data exploration
1. Full models:
    - **Global** and **local** measures
    - Topological attributes. (No. dead ends , average street lenght, average campus area/perimeter...)
1. Selection of certain local areas - to discuss/agree with Mahdi.
    - **Global** and **local** measures
    - This could be created on the basis of: 
        - Surrounding/buffer areas from selected Campus categories
        - Political breaks of the city - In which case we need to get such boundaries.
1. Summarize information into tables
1. Plot and visualize the correspondant/complementary maps.
    - This will greatly differ depending on the number of them.